<a href="https://colab.research.google.com/github/igt-cb/sfdx-scanner/blob/main/panel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import panel as pn

# Define the main dashboard layout
dashboard = pn.Column(sizing_mode='stretch_both', align='center')

# Create a container card that wraps the entire dashboard
container_card = pn.Card(dashboard, title="Ultimate Command Center", background='#f0f0f0')

# Create a header card
header_card = pn.Card(
    pn.Row(
        pn.pane.PNG('logo.png', width=100, height=100, link_url='https://example.com'),
        pn.pane.Markdown('# Welcome to the Ultimate Command Center'),
    ),
    background='#f0f0f0',
    sizing_mode='stretch_width',
)

# Create a sidebar card with various widgets
sidebar_items = pn.WidgetBox(
    pn.pane.Markdown('## Sidebar'),
    pn.widgets.TextInput(name='Search', value=''),
    pn.widgets.Checkbox(name='Option 1', value=False),
    pn.widgets.Checkbox(name='Option 2', value=True),
)

sidebar_card = pn.Card(
    sidebar_items,
    background='#f9f9f9',
    sizing_mode='stretch_width',
)

sidebar_card2 = pn.Card(
    sidebar_items,
    background='#f9f9f9',
    sizing_mode='stretch_width',
)

sidebar_card3 = pn.Card(
    sidebar_items,
    background='#f9f9f9',
    sizing_mode='stretch_width',
)
# Create a main panel with various components
main_panel = pn.Column(
    pn.pane.Markdown('## Main Panel'),
    pn.indicators.Gauge(value=75, bounds=(0, 100), height=325,width=325, align='center'),
    sizing_mode='stretch_width',
)

# Create an additional details card
details_card = pn.Card(
    pn.Column(
        pn.pane.Markdown('## Additional Details'),
        pn.pane.Markdown('Lorem ipsum dolor sit amet, consectetur adipiscing elit.'),
    ),
    background='#f9f9f9',
    sizing_mode='stretch_both',
    scrollable=True,
    height=500,
)

# Create a footer card
footer_card = pn.Card(
    pn.pane.Markdown('© 2022 Ultimate Command Center. All rights reserved.'),
    background='#f0f0f0',
    sizing_mode='stretch_width',
)

# Add the components to the dashboard layout
dashboard.append(header_card)
dashboard.append(pn.Row(sidebar_card,sidebar_card,sidebar_card, main_panel, sizing_mode='stretch_both'))
dashboard.append(details_card)
dashboard.append(footer_card)

# Display the dashboard
container_card.servable()

<ipython-input-48-568cde5d04d9>:7: PanelDeprecationWarning: "Card(..., background='#f0f0f0')" is deprecated and will be removed in version 1.3, use "Card(..., styles={'background': '#f0f0f0'})" instead.
  container_card = pn.Card(dashboard, title="Ultimate Command Center", background='#f0f0f0')
<ipython-input-48-568cde5d04d9>:10: PanelDeprecationWarning: "Card(..., background='#f0f0f0')" is deprecated and will be removed in version 1.3, use "Card(..., styles={'background': '#f0f0f0'})" instead.
  header_card = pn.Card(
<ipython-input-48-568cde5d04d9>:27: PanelDeprecationWarning: "Card(..., background='#f9f9f9')" is deprecated and will be removed in version 1.3, use "Card(..., styles={'background': '#f9f9f9'})" instead.
  sidebar_card = pn.Card(
<ipython-input-48-568cde5d04d9>:33: PanelDeprecationWarning: "Card(..., background='#f9f9f9')" is deprecated and will be removed in version 1.3, use "Card(..., styles={'background': '#f9f9f9'})" instead.
  sidebar_card2 = pn.Card(
<ipython-input

Card(background='#f0f0f0', styles={'background': '#f0f0f0'}, title='Ultimate Command Center')
    [0] Column(align='center', sizing_mode='stretch_both')
        [0] Card(background='#f0f0f0', sizing_mode='stretch_width', styles={'background': '#f0f0f0'})
            [0] Row
                [0] PNG(str, height=100, link_url='https://example.com', sizing_mode='fixed', width=100)
                [1] Markdown(str)
        [1] Row(sizing_mode='stretch_both')
            [0] Card(background='#f9f9f9', sizing_mode='stretch_width', styles={'background': '#f9f9f9'})
                [0] WidgetBox
                    [0] Markdown(str)
                    [1] TextInput(name='Search')
                    [2] Checkbox(name='Option 1')
                    [3] Checkbox(name='Option 2', value=True)
            [1] Card(background='#f9f9f9', sizing_mode='stretch_width', styles={'background': '#f9f9f9'})
                [0] WidgetBox
                    [0] Markdown(str)
                    [1] TextInput(name='Search')
                    [2] Checkbox(name='Option 1')
                    [3] Checkbox(name='Option 2', value=True)
            [2] Card(background='#f9f9f9', sizing_mode='stretch_width', styles={'background': '#f9f9f9'})
                [0] WidgetBox
                    [0] Markdown(str)
                    [1] TextInput(name='Search')
                    [2] Checkbox(name='Option 1')
                    [3] Checkbox(name='Option 2', value=True)
            [3] Column(sizing_mode='stretch_width')
                [0] Markdown(str)
                [1] Markdown(str)
                [2] Gauge(align='center', height=325, value=75, width=125)
        [2] Card(background='#f9f9f9', min_height=500, sizing_mode='stretch_both', styles={'background': '#f9f9f9'})
            [0] Column
                [0] Markdown(str)
                [1] Markdown(str)
        [3] Card(background='#f0f0f0', sizing_mode='stretch_width', styles={'background': '#f0f0f0'})
            [0] Markdown(str)